In [1]:
# Load packages and read in raw data

import pandas as pd
import numpy as np

dete = pd.read_csv('../dete-exit-survey-january-2014.csv', 
                    na_values='Not Stated' #replace 'Not Stated' values with nulls
                  )
tafe = pd.read_csv('../tafe-employee-exit-survey-access-database-december-2013.csv', 
                   encoding='iso-8859-1'
                  )

# Remove unnecessary columns

dete_updated = dete.drop(dete.columns[28:49], axis=1)
tafe_updated = tafe.drop(tafe.columns[17:66], axis=1)

In [2]:
# Set column and row display maximums

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

## Research Questions
- Are employees who only worked for the institutes for a short period of time resigning due to some kind of dissatisfaction? What about employees who have been there longer?
- Are younger employees resigning due to some kind of dissatisfaction? What about older employees?

In [3]:
# Standardize dete_updated column names

dete_updated.columns = dete_updated.columns.str.lower().str.strip().str.replace(' ', '_').str.replace('/', '_')
dete_updated.rename({'separationtype':'separation_type'}, inplace=True, axis=1)

In [4]:
# Standardize tafe_updated column names

pattern = r'[-\.?()/]'
tafe_updated.columns = tafe_updated.columns.str.replace(pattern, '').str.lower().str.strip()
tafe_updated.columns = tafe_updated.columns.str.replace('\s+', ' ').str.replace(' ', '_')

In [5]:
tafe_column_name_map = {'record_id':'id',
                        'reason_for_ceasing_employment':'separation_type',
                        'cessation_year':'cease_date',
                        'workarea':'position',
                        'employment_type_employment_type':'employment_status',
                        'contributing_factors_career_move_public_sector':'career_move_to_public_sector',
                        'contributing_factors_career_move_private_sector':'career_move_to_private_sector',
                        'contributing_factors_interpersonal_conflict':'interpersonal_conflicts',
                        'contributing_factors_job_dissatisfaction':'job_dissatisfaction',
                        'contributing_factors_maternityfamily': 'maternity_family',
                        'contributing_factors_ill_health':'ill_health',
                        'gender_what_is_your_gender':'gender',
                        'currentage_current_age': 'age',
                        'lengthofserviceoverall_overall_length_of_service_at_institute_in_years': 'length_of_service'}

tafe_updated = tafe_updated.rename(tafe_column_name_map, axis=1)
tafe_updated.drop(['institute', 
                   'contributing_factors_career_move_selfemployment', 
                   'contributing_factors_dissatisfaction',
                   'contributing_factors_study',
                   'contributing_factors_travel',
                   'contributing_factors_other',
                   'contributing_factors_none',
                   'classification_classification',
                   'lengthofservicecurrent_length_of_service_at_current_workplace_in_years'], axis=1, inplace=True)

In [6]:
dete_updated.drop(['role_start_date',
                   'classification',
                   'region',
                   'business_unit',
                   'dissatisfaction_with_the_department',
                   'physical_work_environment',
                   'lack_of_recognition',
                   'lack_of_job_security',
                   'work_location',
                   'employment_conditions',
                   'relocation',
                   'study_travel',
                   'traumatic_incident',
                   'work_life_balance',
                   'workload',
                   'none_of_the_above',
                   'aboriginal',
                   'torres_strait',
                   'south_sea',
                   'disability',
                   'nesb'], axis=1, inplace=True)

In [7]:
# Standardize date formate of cease_date in dete

dete_updated['cease_date'] = dete_updated['cease_date'].str.replace(r'[0-9]{2}/', '')

In [8]:
# Create length of service column
dete_updated['cease_date'] = pd.to_numeric(dete_updated['cease_date'])
dete_updated['length_of_service'] = dete_updated['cease_date'] - dete_updated['dete_start_date']

In [9]:
# Change resignation reasons to single type - Resignation

dete_updated.loc[dete_updated['separation_type'].str.contains('Resignat'), 'separation_type'] = 'Resignation'
dete_updated.loc[dete_updated['separation_type'].str.contains('Retirement'), 'separation_type'] = 'Retirement'

In [16]:
tafe_updated.length_of_service.value_counts()

Less than 1 year      147
1-2                   102
3-4                    96
11-20                  89
More than 20 years     71
5-6                    48
7-10                   43
Name: length_of_service, dtype: int64

In [38]:
# Create function to convert float to categorical variable

def create_year_categories(element):
    if element == 0:
        return 'Less than 1 year'
    elif (element > 0) & (element < 3):
        return '1-2'
    elif (element > 2) & (element < 5):
        return '3-4'
    elif (element > 4) & (element < 7):
        return '5-6'
    elif (element > 6) & (element < 11):
        return '7-10'
    elif (element > 10) & (element < 21):
        return '11-20'
    elif element > 20:
        return 'More than 20 years'
    
    
create_year_categories(21)

'More than 20 years'

In [39]:
# Convert float to categorical variable

dete_updated['length_of_service'] = dete_updated['length_of_service'].apply(create_year_categories)


More than 20 years    301
11-20                 144
7-10                   64
5-6                    61
3-4                    59
1-2                    59
Less than 1 year       31
Name: length_of_service, dtype: int64